In [2]:
training_set = [["me free lottery", 1], 
["free get free you", 1],
["you free scholarship", 0],
["free to contact me", 0],
["you won award", 0],
["you ticket lottery", 1]]
                

In [16]:
wordfreq = defaultdict(lambda : [0, 0])
# wordfreq = {}
print(wordfreq["award"])


wordfreq2 = {}
wordfreq2["award"] = [0, 0]
wordfreq2["award"]

[0, 0]


[0, 0]

In [5]:
#spam : 1, normal : 0
# dict = {"award": [1, 0], ...
# cnt_0 (normal)
# cnt_1 (spam)

from collections import defaultdict

normal_cnt  = 0
spam_cnt  = 0

wordfreq = defaultdict(lambda : [0, 0])
for doc, label in training_set:
    words = doc.split()
    for word in words:
        wordfreq[word][label] += 1
        
print(wordfreq)

for key, (cnt0, cnt1) in wordfreq.items():
    normal_cnt += cnt0
    spam_cnt += cnt1

print(normal_cnt)
print(spam_cnt)



defaultdict(<function <lambda> at 0x000001B96870F9D8>, {'me': [1, 1], 'free': [2, 3], 'lottery': [0, 2], 'get': [0, 1], 'you': [2, 2], 'scholarship': [1, 0], 'to': [1, 0], 'contact': [1, 0], 'won': [1, 0], 'award': [1, 0], 'ticket': [0, 1]})
10
10


In [6]:

#wordprob = {"award": [0.1364, 0.0455], ...
k = 0.5
wordprob = defaultdict(lambda : [0, 0])

for key, (cnt0, cnt1) in wordfreq.items():
    wordprob[key][0] = (k + cnt0 ) / ( 2*k + normal_cnt)
    wordprob[key][1] = (k + cnt1 ) / ( 2*k + spam_cnt)
    
wordprob



defaultdict(<function __main__.<lambda>()>,
            {'me': [0.13636363636363635, 0.13636363636363635],
             'free': [0.22727272727272727, 0.3181818181818182],
             'lottery': [0.045454545454545456, 0.22727272727272727],
             'get': [0.045454545454545456, 0.13636363636363635],
             'you': [0.22727272727272727, 0.22727272727272727],
             'scholarship': [0.13636363636363635, 0.045454545454545456],
             'to': [0.13636363636363635, 0.045454545454545456],
             'contact': [0.13636363636363635, 0.045454545454545456],
             'won': [0.13636363636363635, 0.045454545454545456],
             'award': [0.13636363636363635, 0.045454545454545456],
             'ticket': [0.045454545454545456, 0.13636363636363635]})

In [21]:
import math



def get_prob(doc):
    tokens = doc.split()

    log_prob0, log_prob1 = 0.0, 0.0

    for word, (prob0, prob1) in wordprob.items():
        if word in tokens:
            log_prob0 += math.log(prob0)
            log_prob1 += math.log(prob1)

    log_prob0 += math.log(normal_cnt / (normal_cnt + spam_cnt))
    log_prob1 += math.log(spam_cnt / (normal_cnt + spam_cnt))

    prob0 = math.exp(log_prob0)
    prob1 = math.exp(log_prob1)
    
    return prob0, prob1


doc = "scholarship ticket free"
prob0, prob1 = get_prob(doc)


In [22]:
print("정상 확률 {:.1f}%".format((prob0 / (prob0 + prob1)) * 100))
print("스팸 확률 {:.1f}%".format((prob1 / (prob0 + prob1)) * 100))

정상 확률 41.7%
스팸 확률 58.3%


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", sep="\t")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [2]:
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199992 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        199992 non-null  int64 
 1   document  199992 non-null  object
 2   label     199992 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [3]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [9]:
from sklearn.model_selection import train_test_split

X_train, y_train, X_test, y_test = train_test_split(df['document'], df['label'], shuffle=True, random_state=42)
X_train[:10]

186420    진짜요즘런닝맨 개재미없네요 쓸떼없이 게스트 쳐부르지말고 그냥 멤버끼리만하면안됨?? ...
42641                너무 재밌고 감동 ㅠㅠ 그루 츤데레 그리고 미니언들 너무너무너무귀엽다
110059                    스토리는 천군보다 나은데....연출이나 연기 참 못한다...
66757                                  분위기에 취해 버리게 만드는 영화..
164593                                           노잼노잼노잼노잼노잼
56451                               이렇게재밌는영화를보면리뷰를남기는게예의겟지?
130924                                        아.. 어쩌란 말이냐..
26836                                세실리아와 로비의 사랑이 너무 안타깝네요
67863                                        두 배우 넘 조합이 좋다!
145657                                         말이 필요없는 쓰레기.
Name: document, dtype: object

In [5]:
y_train[:10]

189879                                       모피너무많이입고나와 불쾌함
87482                        결말이 ㅠ ㅠ 멜비푸포 완전멋져요 ㅠ이제서야알다니 ㅠㅠ
36369                                 타케모토 노보루 감독, 최고의 걸작이다
82006       진짜 인생 애니메이션 어릴땐 상상력자극으로 재밌고 커서는 우정간의 감동을 보며 재밌고
89163     몇몇 맞지않는 전개도 있지만 이 정도면 훌륭하다. 우리 이순신장군님 영화도 이렇게 ...
79195     번개치고 귀신튀어나오는 성궤나 불사의 영약 성배나 고대 중남미문명 외계인 개입설이나...
94218     우리나라 몇 안되는 진행 몰입도 좋은 영화중 하나. 2편이 기대되는데 안나올듯 하다...
124006    네러티브는 하나도 없는 예쁜 음악 동영상. 취미로 영화만드는 것은 이걸로 끝나길 ....
27053                              나이를 먹을수록 가슴깊이 느낄 수 있는 영화
82529                         진ㅉ ㅏ잼잇따!!!!!!!후끈후끈화끈화끈 액숀!!!!
Name: document, dtype: object

In [6]:
count_vect = CountVectorizer()
X_train_count = count_vect.fit_transform(X_train)
clf = MulitinomialNB().fit(X_train_count, y_train)

NameError: name 'CountVectorizer' is not defined

In [7]:
X_text_count = count_vect.transform(X_test)
predicted = clf.predict(X_text_count)
np.mean(predicted == y_test)

NameError: name 'count_vect' is not defined